In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [58]:
df = pd.read_csv("data/VisaPremier.txt", delimiter="\t")

In [12]:
count_nan = len(df) - df.count()

In [74]:
df.head(10)

,matricul,departem,ptvente,sexe,age,sitfamil,anciente,csp,codeqlt,nbimpaye,...,mtbon,nbpaiecb,nbcb,nbcbptar,avtscpte,aveparfi,cartevp,sexer,cartevpr,nbjdebit
0,148009,31,1,Shom,51,Fmar,238,Pcad,A,0,...,0,14,2,0,1303700,556967,Coui,0,1,1
1,442153,82,6,Shom,52,Fmar,270,Pcad,A,0,...,19500000,5,2,0,19856243,133896,Coui,0,1,0
2,552427,97,1,Shom,58,Fmar,139,Pcad,C,0,...,0,0,1,0,122745,0,Coui,0,1,0
3,556005,40,1,Shom,27,Fcel,99,Psan,B,0,...,0,14,2,0,83224,0,Coui,0,1,0
4,556686,65,1,Shom,49,Fsep,89,Pemp,A,0,...,0,11,3,1,494773,21423,Coui,0,1,15
5,642680,65,1,Shom,64,Fmar,216,Pcad,A,0,...,0,27,1,0,81218,0,Coui,0,1,3
6,643508,65,1,Shom,38,Fcel,188,Pemp,C,0,...,0,13,2,0,14706,0,Coui,0,1,13
7,808224,31,1,Sfem,46,Fmar,135,Pcad,A,0,...,0,27,3,0,105333,0,Coui,1,1,35
8,941007,31,1,Shom,45,Fmar,393,Pouv,A,0,...,0,15,2,0,310878,1452,Coui,0,1,0
9,981681,31,1,Shom,38,Fcel,63,Pcad,D,0,...,0,.,0,0,158955,149358,Coui,0,1,31


In [73]:
df.dtypes[df.dtypes == "object"]

departem    object
sexe        object
sitfamil    object
csp         object
codeqlt     object
agemvt      object
nbpaiecb    object
cartevp     object
dtype: object

In [40]:
#df.codeqlt.value_counts()

matricul     int64
departem    object
ptvente      int64
sexe        object
age          int64
sitfamil    object
anciente     int64
csp         object
codeqlt     object
nbimpaye     int64
mtrejet      int64
nbopguic     int64
moycred3     int64
aveparmo     int64
endette      int64
engagemt     int64
engagemc     int64
engagemm     int64
nbcptvue     int64
moysold3     int64
moycredi     int64
agemvt      object
nbop         int64
mtfactur     int64
engageml     int64
nbvie        int64
mtvie        int64
nbeparmo     int64
mteparmo     int64
nbeparlo     int64
mteparlo     int64
nblivret     int64
mtlivret     int64
nbeparlt     int64
mteparlt     int64
nbeparte     int64
mteparte     int64
nbbon        int64
mtbon        int64
nbpaiecb    object
nbcb         int64
nbcbptar     int64
avtscpte     int64
aveparfi     int64
cartevp     object
sexer        int64
cartevpr     int64
nbjdebit     int64
dtype: object

In [ ]:
y = df.cartevpr
X = df.drop("cartevpr", axis=1)

In [55]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from sklearn.ensemble import ExtraTreesClassifier


##get_ipython().run_line_magic('matplotlib', 'inline')

file = r'RCM_Binary.csv'
data = pd.read_csv("data/VisaPremier.txt", delimiter="\t")
print("data loaded successfully ...")

# Define features and target
y = df.cartevpr
X = df.drop("cartevpr", axis=1)

#split to training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

# define classifier and fitting data
forest = ExtraTreesClassifier(random_state=1)
forest.fit(X_train, y_train)

# predict and get confusion matrix
y_pred = forest.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

#Applying 10-fold cross validation
accuracies = cross_val_score(estimator=forest, X=X_train, y=y_train, cv=10)
print("accuracy (10-fold): ", np.mean(accuracies))

# Features importances
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
feature_list = [X.columns[indices[f]] for f in range(X.shape[1])]  #names of features.
ff = np.array(feature_list)

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f) name: %s" % (f + 1, indices[f], importances[indices[f]], ff[indices[f]]))


# Plot the feature importances of the forest
plt.figure()
plt.rcParams['figure.figsize'] = [16, 6]
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), ff[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()


## The new additions to get feature importance to classes: 

# To get the importance according to each class:
def class_feature_importance(X, Y, feature_importances):
    N, M = X.shape
    X = scale(X)

    out = {}
    for c in set(Y):
        out[c] = dict(
            zip(range(N), np.mean(X[Y==c, :], axis=0)*feature_importances)
        )

    return out

result = class_feature_importance(X, y, importances)
print (json.dumps(result,indent=4))

# Plot the feature importances of the forest

titles = ["Did not Divert", "Diverted"]
for t, i in zip(titles, range(len(result))):
    plt.figure()
    plt.rcParams['figure.figsize'] = [16, 6]
    plt.title(t)
    plt.bar(range(len(result[i])), result[i].values(),
           color="r", align="center")
    plt.xticks(range(len(result[i])), ff[list(result[i].keys())], rotation=90)
    plt.xlim([-1, len(result[i])])
    plt.show()

/home/yannis/.local/lib/python2.7/site-packages/sklearn/utils/__init__.py:12: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .murmurhash import murmurhash3_32
/home/yannis/.local/lib/python2.7/site-packages/sklearn/utils/extmath.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._logistic_sigmoid import _log_logistic_sigmoid
/home/yannis/.local/lib/python2.7/site-packages/sklearn/utils/extmath.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .sparsefuncs_fast import csr_row_norms
/home/yannis/.local/lib/python2.7/site-packages/sklearn/metrics/cluster/supervised.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .expected_mutual_info_fast import expected_mutual_information
/home/yannis/.local/lib/python2.7/site-packages/sklearn/metrics/pairwis

data loaded successfully ...


ValueError: could not convert string to float: Cnon